<a id="A"></a>
## Partie A : Analyse des risques de crédit

In [ ]:
# Importer les bibliothèques nécessaires pour la partie A
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

import time
import warnings
warnings.filterwarnings('ignore')

# Afficher toutes les colonnes. Ne cachez rien.
pd.set_option('display.max_columns', None)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<br>
<a id="A1"></a>
<hr style="border:1px solid black"> </hr>
<h1 style="font-size:18px; font-weight:bold;"> Tâche A-1 : Lecture et préparation des ensembles de données</h1>
<hr style="border:1px solid black"> </hr>

In [ ]:
#Contrairement à l'ensemble de données d'examen, les valeurs TARGET ne sont pas disponibles pour les données de test.
#Par conséquent, le fichier global "traintest" est dérivé du fichier application_train.csv.
dfapptraintest_raw = pd.read_csv('/kaggle/input/projet7/application_train.csv')
dfprevapp_raw = pd.read_csv('/kaggle/input/projet7/previous_application.csv')
dfbureau_raw  = pd.read_csv('/kaggle/input/projet7/bureau.csv')


In [ ]:
# Afficher toutes les colonnes. Ne cachez rien.
pd.set_option('display.max_columns', None)

In [ ]:
dfapptraintest_raw.head()

In [ ]:
dfapptraintest_raw['FLAG_DOCUMENT_21'].unique

In [ ]:
dfprevapp_raw.head()

In [ ]:
# Utilisez la méthode intersection des colonnes pour trouver les colonnes communes
common_columns = list(set(dfapptraintest_raw.columns) & set(dfprevapp_raw.columns) & set(dfbureau_raw.columns))

# Affichez les colonnes communes
print("Colonnes communes entre les trois DataFrames :", common_columns)

In [ ]:
#Sélectionner les colonnes (réduction aux colonnes de l'ensemble de données d'examen) :
dfapptraintest_raw = dfapptraintest_raw.iloc[:, np.r_[0:32, 34:44, 95, 116:122]]
dfprevapp_raw = dfprevapp_raw.iloc[:, np.r_[1:8, 16:23, 28:30, 36]]
dfbureau_raw = dfbureau_raw.iloc[:, np.r_[0, 2, 4:14, 15:17]]

# Aperçu du nombre de lignes et de colonnes des ensembles de données
print('shape of dfapptraintest_raw:', dfapptraintest_raw.shape)
print('shape of dfprevapp_raw:', dfprevapp_raw.shape)
print('shape of dfbureau_raw:', dfbureau_raw.shape)

In [ ]:
# Train-Test-Split:  Répartition aléatoire des données de demande en données d'entraînement et de test.
np.random.seed(111)  # Définition du générateur de nombres aléatoires pour la reproductibilité
dfapptraintest_raw['TYPE'] = np.random.choice(['TRAIN', 'TEST'], size=len(dfapptraintest_raw), p=[0.75, 0.25])
dfapptrain_raw = dfapptraintest_raw[dfapptraintest_raw['TYPE'] == 'TRAIN']
dfapptest_raw = dfapptraintest_raw[dfapptraintest_raw['TYPE'] == 'TEST']
print('shape of dfapptrain_raw:', dfapptrain_raw.shape)
print('shape of dfapptest_raw:', dfapptest_raw.shape)

<br>
<a id="A2"></a>
<hr style="border:1px solid black"> </hr>
<h1 style="font-size:18px; font-weight:bold;">Tâche A-2 : Analyse des ensembles de données</h1>
<hr style="border:1px solid black"> </hr>

### a) Ensemble de données combiné avec les données d'entraînement et de test.

In [ ]:
print("Nombre de lignes et de colonnes.:", dfapptraintest_raw.shape)
dfapptraintest_raw.head()

In [ ]:
# Vérificatio des types des données
pd.DataFrame(dfapptraintest_raw.dtypes).T

In [ ]:
# Description des caractéristiques catégorielles.
dfapptraintest_raw.describe(include=['object'])

In [ ]:
# Ajustements des types de données.
dfapptraintest = dfapptraintest_raw.copy(deep=True)

# Conformément aux directives, les variables suivantes sont considérées comme catégorielles.
cat = ['TARGET', 'NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_TYPE_SUITE','NAME_INCOME_TYPE',
       'NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE','ORGANIZATION_TYPE',
       'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
       'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY','REG_CITY_NOT_WORK_CITY','LIVE_CITY_NOT_WORK_CITY','FLAG_MOBIL',
       'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'TYPE']
dfapptraintest[cat]=dfapptraintest[cat].astype('category')

# Vérification des types de données ajustés.
pd.DataFrame(dfapptraintest.dtypes).T

In [ ]:
#La fonction print_column_datatypes affiche, pour un DataFrame donné, le nombre de colonnes avec des valeurs numériques,
#le nombre de colonnes avec des valeurs catégorielles, ainsi que le nombre total de colonnes.
def print_column_datatypes(data):
    categorical = ['category']
    numerical = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    print("Columns with numerical values: " + str(len(data.select_dtypes(include=numerical).columns.tolist())))
    print("Columns with categorical values: " + str(len(data.select_dtypes(include=categorical).columns.tolist())))
    print("Total " + str(len(data.select_dtypes(include=numerical).columns.tolist()) + len(data.select_dtypes(include=categorical).columns.tolist())) + " columns")

print_column_datatypes(dfapptraintest.loc[:, dfapptraintest.columns != 'TYPE'])

In [ ]:
# La fonction `nan_df_create` crée un autre Dataframe pour un Dataframe donné, composé de tous les noms de colonnes et de leurs proportions
 #(en pourcentage) de valeurs manquantes, seuls les noms de colonnes avec au moins une valeur manquante sont inclus.

def nan_df_create(data):
    nan_percentages = data.isna().sum() * 100 / len(data)
    df = pd.DataFrame({'Column': nan_percentages.index, 'Percentage': nan_percentages.values})
    df = df[df['Percentage'] > 0]
    df.sort_values(by='Percentage', ascending=False, inplace=True)
    return df

# Calculer et afficher la proportion de valeurs manquantes (NaN)
df = nan_df_create(dfapptraintest[dfapptraintest['TYPE'] == 'TRAIN'])
print(df.to_string())


### b) Données de previous_application

In [ ]:
print("Number of rows and columns:", dfprevapp_raw.shape)
dfprevapp_raw.head()


In [ ]:
#Vérifier les types de données
pd.DataFrame(dfprevapp_raw.dtypes).T

In [ ]:
# Description des caractéristiques nominales.
dfprevapp_raw.describe(include=['object'])

In [ ]:
#Ajustements aux types de données
dfprevapp = dfprevapp_raw.copy(deep=True)

#Conformément aux directives, les variables suivantes sont considérées comme catégorielles
cat = ['NAME_CONTRACT_TYPE','NAME_CONTRACT_STATUS','NAME_PAYMENT_TYPE','CODE_REJECT_REASON','NAME_TYPE_SUITE','NAME_CLIENT_TYPE',
       'NAME_GOODS_CATEGORY','NAME_YIELD_GROUP','NFLAG_INSURED_ON_APPROVAL']
dfprevapp[cat]=dfprevapp[cat].astype('category')

#Contrôle des types de données ajustés
pd.DataFrame(dfprevapp.dtypes).T

In [ ]:
# Nombre de colonnes numériques et catégorielles.
print_column_datatypes(dfprevapp)

In [ ]:
# Calculer et afficher la proportion de valeurs manquantes (NaN).
df = nan_df_create(dfprevapp)
print(df.to_string())

### c) Données du dataset bureau

In [ ]:
print("Nombre de lignes et de colonnes : ", dfbureau_raw.shape)
dfbureau_raw.head()

In [ ]:
# Vérifier les types des données
pd.DataFrame(dfbureau_raw.dtypes).T

In [ ]:
# Description des caractéristiques catégorielles
dfprevapp_raw.describe(include=['object'])

In [ ]:
# Ajustements aux types de données
dfbureau = dfbureau_raw.copy(deep=True)

# Selon les spécifications, les variables suivantes sont considérées comme catégorielles.
cat = ['CREDIT_ACTIVE']
dfbureau[cat]=dfbureau[cat].astype('category')

# Nombre de colonnes numériques et catégorielles.
print_column_datatypes(dfbureau)

In [ ]:
# Calculer et afficher la proportion de valeurs manquantes (NaN).
df = nan_df_create(dfbureau)
print(df.to_string())

Pour les deux ensembles de données dfprevapp et dfbureau, l'occurrence des ID respectifs est comptée.

In [ ]:
dfprevapp['SK_ID_CURR'].value_counts().head(3)

In [ ]:
dfbureau['SK_ID_CURR'].value_counts().head(3)

  **Interprétation** : Il y a plusieurs entrées par ID à la fois dans la table previous_application et dans la table bureau. Ces entrées doivent être agrégées de manière appropriée avant d'être fusionnées avec les ensembles de données application.

In [ ]:
duplicates = dfprevapp[dfprevapp['SK_ID_CURR'].duplicated()]

num_duplicates = len(duplicates)
if num_duplicates > 0:
    print("Nombre de lignes avec des doublons de SK_ID_CURR :", num_duplicates)
    print("Lignes avec des doublons de SK_ID_CURR :")
    print(duplicates)
else:
    print("Aucun doublon de SK_ID_CURR trouvé.")


<br>
<a id="A3"></a>
<hr style="border:1px solid black"> </hr>
<h1 style="font-size:18px; font-weight:bold;">Tâche A-3 : Visualisation et préparation des ensembles de données </h1>
<hr style="border:1px solid black"> </hr>

### Partie a)

In [ ]:
dfapptrain = dfapptraintest[dfapptraintest['TYPE'] == 'TRAIN']
dfapptrain = dfapptrain.loc[:, dfapptrain.columns != 'TYPE']

In [ ]:
#La fonction plot_cat génère un graphique à barres empilées pour chaque caractéristique catégorielle
#d'un DataFrame donné, en fonction des valeurs d'une colonne cible donnée.
def plot_cat(data, target, n_cols, n_rows, title):
    cols = data.select_dtypes(include='category').columns.tolist()
    fg,ax = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(n_rows*7, n_cols*5), squeeze=False)
    plt.suptitle(title, fontsize=28, y=0.95)
    j=0;
    i=0;

    for col in cols:
        data.groupby([col, target]).size().unstack().fillna(0).plot(kind='bar', stacked=True, color=['steelblue', 'red'], ax=ax[i][j])
        i += 1
        if(i>=n_rows):
            i = 0
            j += 1

    # Supprimez les axes qui ne sont pas nécessaires (ici, uniquement pour le cas particulier où la dernière colonne contient des graphiques vides).
    while (i<n_rows):
        fg.delaxes(ax[i,j])
        i += 1

plot_cat(dfapptrain, 'TARGET', 6, 5, "Comptages graphiques (countplots) pour les variables catégorielles dans l'ensemble de données application_train.")

In [ ]:
#La fonction plot_cat_percent génère, pour un DataFrame donné, un graphique à barres empilées pour chaque
#caractéristique catégorielle du DataFrame, en fonction des valeurs d'une colonne cible donnée.
#Ces graphiques se complètent à 100% pour chaque niveau de la caractéristique catégorielle.
def plot_cat_percent(data, target, n_cols, n_rows, title):
    cols = data.select_dtypes(include='category').columns.tolist()
    fg,ax = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(n_rows*7, n_cols*5), squeeze=False)
    plt.suptitle(title, fontsize=28, y=0.95)
    j=0;
    i=0;

    for col in cols:
        (data.groupby([col, target]).size()/data.groupby([col]).size()).unstack().fillna(0).plot(kind='bar', stacked=True, color=['steelblue', 'red'], ax=ax[i][j])
        i += 1
        if(i>=n_rows):
            i = 0
            j += 1

    #Supprime les axes qui ne sont pas nécessaires (ici, uniquement pour le cas spécial où la dernière colonne contient des graphiques vides)
    while (i<n_rows):
        fg.delaxes(ax[i,j])
        i += 1

plot_cat_percent(dfapptrain, 'TARGET', 6, 5, "Graphiques à barres empilées en pourcentage pour les variables catégorielles dans l'ensemble de données de application_train")

__Interprétation:__

À partir des graphiques, les observations suivantes peuvent être faites (sélection) :

La variable cible TARGET n'est pas équilibrée : il y a nettement moins de prêts en défaut que de prêts non en défaut.

Les individus masculins semblent avoir une probabilité plus élevée de défaut de prêt que les individus féminins.

Pour la caractéristique OCCUPATION_TYPE, la classe W semble avoir une probabilité de défaut plus élevée que les autres classes.

Pour la caractéristique NAME_EDUCATION_TYPE, la classe L semble avoir une probabilité de défaut plus élevée que les autres classes.

### Partie b)

In [ ]:
#La fonction plot_num génère, pour un DataFrame donné, un tracé de densité de noyau différencié
#en fonction des valeurs d'une colonne cible donnée, pour chaque caractéristique numérique du DataFrame.
def plot_num(data, target, n_cols,n_rows, title):
    cols = data.select_dtypes(exclude='category').columns.tolist()
    cols.remove('SK_ID_CURR')
    fg,ax = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(3*n_rows, 18*n_cols), squeeze=False)
    plt.suptitle(title, fontsize=18, y=0.95)
    j=0;
    i=0;
    for col in cols:
        sns.kdeplot(data.loc[data['TARGET'] == 1, col], label='target == 1', color='r', ax=ax[i,j])
        sns.kdeplot(data.loc[data['TARGET'] == 0, col], label='target == 0', color='b', ax=ax[i,j])
        i += 1
        if(i>=n_rows):
            i = 0
            j += 1

plot_num(dfapptrain, 'TARGET', 2, 11, "Variables numériques dans l'ensemble de données application_train")

__Interprétation:__

À partir des graphiques, les observations suivantes peuvent être faites :

* Les scores (EXT_SOURCE_1, EXT_SOURCE_2 et EXT_SOURCE_3) semblent déjà offrir une bonne différenciation.

* La caractéristique DAYS_BIRTH semble également offrir une bonne différenciation dans la plage des valeurs à droite.

* Pour la caractéristique DAYS_EMPLOYED, on peut observer une tendance douteuse dans la plage des valeurs à droite. Cela sera analysé plus en détail dans la partie c).

### Parties c) et d)

In [ ]:
#Marquer les valeurs abérrantes (1000 ans) comme manquantes.
dfapptrain['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
dfapptraintest['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)

# Pour les colonnes spécifiées, les valeurs se référant à une période de plus de 50 ans en arrière sont définies à 0.
dfbureau['DAYS_CREDIT_ENDDATE'][dfbureau['DAYS_CREDIT_ENDDATE'] > -50*365] = 0
dfbureau['DAYS_ENDDATE_FACT'][dfbureau['DAYS_ENDDATE_FACT'] > -50*365] = 0
dfbureau['DAYS_CREDIT_UPDATE'][dfbureau['DAYS_CREDIT_UPDATE'] > -50*365] = 0

<br>
<a id="A4"></a>
<hr style="border:1px solid black"> </hr>
<h1 style="font-size:18px; font-weight:bold;">Tâche A-4: Feature Engineering </h1>
<hr style="border:1px solid black"> </hr>

**Dans cette tâche, les informations des ensembles de données previous_application, bureau et l'ensemble de données combiné d'entraînement et de test doivent être préparées.**

**a) Les tâches suivantes doivent être effectuées pour chacun des trois ensembles de données mentionnés :**

- **Pour les caractéristiques numériques, les valeurs manquantes doivent être remplacées par 0. Pour les caractéristiques catégorielles, les valeurs manquantes doivent être remplacées par la nouvelle catégorie "categorie_NaN".**

- **De plus, les caractéristiques spécifiques au domaine indiquées dans l'Annexe 1 doivent être incluses dans l'ensemble de données examiné.**

- **Le jeu de données doit être agrégé de manière à avoir exactement une ligne par ID (selon SK_ID_CURR). Les points suivants doivent être pris en compte :**

    - **Avant l'agrégation, les ensembles de données doivent être triés par SK_ID_CURR et DAYS_DECISION (dans le cas de previous_application), ou par SK_ID_CURR et DAYS_CREDIT_ENDDATE (dans le cas de bureau), en ordre croissant..**

    - **Pour les caractéristiques numériques, les quatre agrégations suivantes doivent être utilisées : sum (somme), mean (moyenne), max (maximum) et min (minimum). Par conséquent, une colonne numérique sera remplacée par quatre colonnes correspondant aux agrégations appropriées..**

    - **Pour les caractéristiques catégorielles, les trois agrégations mod(mode), first (première valeur apparue) et last (dernière valeur apparue) doivent être utilisées. Par conséquent, une colonne catégorielle est remplacée par trois colonnes correspondant aux agrégations respectives.**
    
**b) Quelles sont les considérations générales lors de l'ajout de nouvelles caractéristiques basées sur des connaissances de domaine, similaires à celles présentées à l'Annexe 1 ? Quelles mesures peuvent être prises pour garantir la qualité des caractéristiques additionnelles ?**

In [ ]:
# La fonction mod calcule le mode (= valeur la plus fréquemment rencontrée) des valeurs entrées.
def mod(x):
    m = pd.Series.mode(x);
    return m.values[0] if not m.empty else np.nan

aggregations_num = ['sum', 'median', 'max', 'min']
aggregations_cat = [mod, 'first', 'last']

### 4 a)

### PREVIOUS APPLICATION

Comme indiqué dans la Partie 1), le jeu de données contient les colonnes numériques suivantes avec des valeurs manquantes : AMT_DOWN_PAYMENT, AMT_GOODS_PRICE, AMT_ANNUITY et CNT_PAYMENT. Ces valeurs seront remplacées par 0.

Les variables catégorielles NAME_TYPE_SUITE et NFLAG_INSURED_ON_APPROVAL contiennent également des valeurs manquantes.

In [ ]:
# Remplacement des valeurs manquantes (NaN) dans les colonnes numériques par la moyenne.
numerical_columns = ['AMT_DOWN_PAYMENT','AMT_GOODS_PRICE','AMT_ANNUITY','CNT_PAYMENT','AMT_CREDIT']
dfprevapp[numerical_columns] = dfprevapp[numerical_columns].fillna(0)

# Remplacement des valeurs manquantes (NaN) dans les colonnes catégorielles par la valeur categorie_NaN.
dfprevapp['NAME_TYPE_SUITE'] = dfprevapp['NAME_TYPE_SUITE'].cat.add_categories("categorie_NaN").fillna("categorie_NaN")
dfprevapp['NFLAG_INSURED_ON_APPROVAL'] = dfprevapp['NFLAG_INSURED_ON_APPROVAL'].cat.add_categories("categorie_NaN").fillna("categorie_NaN")

# Vérification des remplacements.
dfprevapp.isnull().sum().sum()

In [ ]:
# En se basant sur les connaissances du domaine, les caractéristiques demandées sont ajoutées.

#Toutes les caractéristiques nouvellement ajoutées (manuellement) reçoivent le préfixe PreDK.
#Pour chaque ligne, déterminer le nombre de valeurs manquantes :
dfprevapp['PreDK_MIssING_VALUES_TOTAL_PREV'] = dfprevapp.isna().sum(axis = 1)

# Différence entre la demande et le crédit réel.
dfprevapp['PreDK_AMT_DECLINED'] = dfprevapp['AMT_APPLICATION'] - dfprevapp['AMT_CREDIT']

# VRatio entre le montant du crédit et la valeur du bien à financer.
dfprevapp['PreDK_AMT_CREDIT_GOODS_RATIO'] = dfprevapp['AMT_CREDIT'] / (dfprevapp['AMT_GOODS_PRICE'] + 0.00001)

# Différence entre le montant du crédit et la valeur du bien à financer.
dfprevapp['PreDK_AMT_CREDIT_GOODS_DIFF'] = dfprevapp['AMT_CREDIT'] - dfprevapp['AMT_GOODS_PRICE']

# Quotient entre le montant du crédit demandé et le montant du crédit accordé.
dfprevapp['PreDK_AMT_CREDIT_APPLICATION_RATIO'] = dfprevapp['AMT_APPLICATION'] / (dfprevapp['AMT_CREDIT'] + 0.00001)

# Estimation des paiements d'intérêts.
dfprevapp['PreDK_AMT_INTEREST'] = dfprevapp['CNT_PAYMENT'] * dfprevapp['AMT_ANNUITY'] - dfprevapp['AMT_CREDIT']

# Quotient entre les mensualités et le montant du crédit.
dfprevapp['PreDK_ANNUITY_CREDIT_RATIO'] = dfprevapp['AMT_ANNUITY'] / (dfprevapp['AMT_CREDIT'] + 0.00001)

In [ ]:
aggregations_for_previous_application = {
            #numerical
            'AMT_ANNUITY' :               aggregations_num,
            'AMT_APPLICATION' :           aggregations_num,
            'AMT_CREDIT' :                aggregations_num,
            'AMT_DOWN_PAYMENT' :          aggregations_num,
            'AMT_GOODS_PRICE' :           aggregations_num,
            'DAYS_DECISION' :             aggregations_num,
            'CNT_PAYMENT' :               aggregations_num,
            #domain knowledge
            'PreDK_MIssING_VALUES_TOTAL_PREV' :    aggregations_num,
            'PreDK_AMT_DECLINED' :                 aggregations_num,
            'PreDK_AMT_CREDIT_GOODS_RATIO' :       aggregations_num,
            'PreDK_AMT_CREDIT_GOODS_DIFF' :        aggregations_num,
            'PreDK_AMT_CREDIT_APPLICATION_RATIO' : aggregations_num,
            'PreDK_AMT_INTEREST' :                 aggregations_num,
            'PreDK_ANNUITY_CREDIT_RATIO':          aggregations_num,
            #categorical
            'NAME_CONTRACT_TYPE' :        aggregations_cat,
            'NAME_CONTRACT_STATUS' :      aggregations_cat,
            'NAME_PAYMENT_TYPE' :         aggregations_cat,
            'CODE_REJECT_REASON' :        aggregations_cat,
            'NAME_TYPE_SUITE' :           aggregations_cat,
            'NAME_CLIENT_TYPE' :          aggregations_cat,
            'NAME_GOODS_CATEGORY' :       aggregations_cat,
            'NAME_YIELD_GROUP' :          aggregations_cat,
            'NFLAG_INSURED_ON_APPROVAL' : aggregations_cat,
            }

In [ ]:
# Tri et réalisation des agrégations en tenant compte de toutes les demandes associées.
tic = time.time()

dfprevapp = dfprevapp.sort_values(by = ['SK_ID_CURR','DAYS_DECISION'])
dfprevapp_agg = dfprevapp.groupby('SK_ID_CURR').agg(aggregations_for_previous_application)

# Correction des noms de colonnes.
dfprevapp_agg.columns = ['_'.join(col) for col in dfprevapp_agg.columns.values]

print("time (sec):" + "%6.0f" % (time.time() - tic))



In [ ]:
# Examiner les résultats.
dfprevapp_agg.head(5)

In [ ]:
dfprevapp_agg.shape

### BUREAU

Ce jeu de données contient des valeurs manquantes dans les colonnes numériques suivantes :
AMT_ANNUITY, AMT_CREDIT_MAX_OVERDUE, DAYS_ENDDATE_FACT, AMT_CREDIT_SUM_LIMIT, AMT_CREDIT_SUM_DEBT, DAYS_CREDIT_ENDDATE et AMT_CREDIT_SUM.
Celles-ci seront remplacées par 0.

In [ ]:
numerical_columns = ['AMT_ANNUITY','AMT_CREDIT_MAX_OVERDUE','DAYS_ENDDATE_FACT','AMT_CREDIT_SUM_LIMIT','AMT_CREDIT_SUM_DEBT','DAYS_CREDIT_ENDDATE','AMT_CREDIT_SUM']
dfbureau[numerical_columns] = dfbureau[numerical_columns].fillna(0)

# Vérification des remplacements.
dfbureau.isnull().sum().sum()

In [ ]:
# En se basant sur les connaissances du domaine, les caractéristiques demandées sont ajoutées.
# Toutes les caractéristiques nouvellement ajoutées (manuellement) porteront le préfixe "BurDK".
dfbureau['BurDK_CREDIT_DURATION'] = np.abs(dfbureau['DAYS_CREDIT'] - dfbureau['DAYS_CREDIT_ENDDATE'])
dfbureau['BurDK_FLAG_OVERDUE_RECENT'] = [0 if ele == 0 else 1 for ele in dfbureau['CREDIT_DAY_OVERDUE']]
dfbureau['BurDK_CURRENT_AMT_OVERDUE_DURATION_RATIO'] = dfbureau['AMT_CREDIT_SUM_OVERDUE'] / (dfbureau['BurDK_CREDIT_DURATION'] + 0.00001)
dfbureau['BurDK_AMT_OVERDUE_DURATION_LEFT_RATIO'] = dfbureau['AMT_CREDIT_SUM_OVERDUE'] / (dfbureau['DAYS_CREDIT_ENDDATE'] + 0.00001)
dfbureau['BurDK_CNT_PROLONGED_MAX_OVERDUE_MUL'] = dfbureau['CNT_CREDIT_PROLONG'] * dfbureau['AMT_CREDIT_MAX_OVERDUE']
dfbureau['BurDK_CNT_PROLONGED_DURATION_RATIO'] = dfbureau['CNT_CREDIT_PROLONG'] / (dfbureau['BurDK_CREDIT_DURATION'] + 0.00001)
dfbureau['BurDK_CURRENT_DEBT_TO_CREDIT_RATIO'] = dfbureau['AMT_CREDIT_SUM_DEBT'] / (dfbureau['AMT_CREDIT_SUM'] + 0.00001)
dfbureau['BurDK_CURRENT_CREDIT_DEBT_DIFF'] = dfbureau['AMT_CREDIT_SUM'] - dfbureau['AMT_CREDIT_SUM_DEBT']
dfbureau['BurDK_AMT_ANNUITY_CREDIT_RATIO'] = dfbureau['AMT_ANNUITY'] / (dfbureau['AMT_CREDIT_SUM'] + 0.00001)
dfbureau['BurDK_CREDIT_ENDDATE_UPDATE_DIFF'] = np.abs(dfbureau['DAYS_CREDIT_UPDATE'] - dfbureau['DAYS_CREDIT_ENDDATE'])

In [ ]:
aggregations_for_bureau = {
            #numerical
            'DAYS_CREDIT' :            aggregations_num,
            'CREDIT_DAY_OVERDUE' :     aggregations_num,
            'DAYS_CREDIT_ENDDATE' :    aggregations_num,
            'DAYS_ENDDATE_FACT' :      aggregations_num,
            'AMT_CREDIT_MAX_OVERDUE' : aggregations_num,
            'CNT_CREDIT_PROLONG':      aggregations_num,
            'AMT_CREDIT_SUM':          aggregations_num,
            'AMT_CREDIT_SUM_DEBT':     aggregations_num,
            'AMT_CREDIT_SUM_LIMIT':    aggregations_num,
            'AMT_CREDIT_SUM_OVERDUE':  aggregations_num,
            'DAYS_CREDIT_UPDATE':      aggregations_num,
            'AMT_ANNUITY':             aggregations_num,
            #Domain knowledge
            'BurDK_CREDIT_DURATION' :                    aggregations_num,
            'BurDK_FLAG_OVERDUE_RECENT' :                aggregations_num,
            'BurDK_CURRENT_AMT_OVERDUE_DURATION_RATIO' : aggregations_num,
            'BurDK_AMT_OVERDUE_DURATION_LEFT_RATIO' :    aggregations_num,
            'BurDK_CNT_PROLONGED_MAX_OVERDUE_MUL' :      aggregations_num,
            'BurDK_CNT_PROLONGED_DURATION_RATIO' :       aggregations_num,
            'BurDK_CURRENT_DEBT_TO_CREDIT_RATIO' :       aggregations_num,
            'BurDK_CURRENT_CREDIT_DEBT_DIFF' :           aggregations_num,
            'BurDK_AMT_ANNUITY_CREDIT_RATIO' :           aggregations_num,
            'BurDK_CREDIT_ENDDATE_UPDATE_DIFF' :         aggregations_num,
            #categorical
            'CREDIT_ACTIVE' : aggregations_cat
            }

In [ ]:
# Trier les données et les regrouper par SK_ID_CURR (identifiant du client).
dfbureau = dfbureau.sort_values(by = ['SK_ID_CURR','DAYS_CREDIT_ENDDATE'])
dfbureau_agg = dfbureau.groupby('SK_ID_CURR').agg(aggregations_for_bureau)

# Corriger les noms de colonnes.
dfbureau_agg.columns = ['_'.join(col) for col in dfbureau_agg.columns.values]

# Examiner les résultats.
dfbureau_agg.head()

### APPLICATION_TRAIN

Les valeurs numériques manquantes sont remplacées par 0.

Les deux caractéristiques catégorielles (OCCUPATION_TYPE et NAME_TYPE_SUITE) ont une catégorie propre.

Dans ce jeu de données, l'ID SK_ID_CURR est unique, donc aucune agrégation n'est effectuée.

In [ ]:
# En plus des colonnes trouvées avec des valeurs NaN dans la tâche A-2, nous devons également effectuer
# une imputation pour la colonne DAYS_EMPLOYED, car nous avions remplacé la valeur 365243 par NaN.

numerical_columns = ['OWN_CAR_AGE','AMT_GOODS_PRICE','AMT_ANNUITY','DAYS_EMPLOYED', 'DAYS_LAST_PHONE_CHANGE'
                     ,'EXT_SOURCE_1','EXT_SOURCE_3','EXT_SOURCE_2'
                     ,'AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK'
                     ,'AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR'
                     ,'CNT_FAM_MEMBERS']
dfapptraintest[numerical_columns] = dfapptraintest[numerical_columns].fillna(0)

dfapptraintest['OCCUPATION_TYPE'] = dfapptraintest['OCCUPATION_TYPE'].cat.add_categories("Kategorie_NaN").fillna("Kategorie_NaN")
dfapptraintest['NAME_TYPE_SUITE'] = dfapptraintest['NAME_TYPE_SUITE'].cat.add_categories("Kategorie_NaN").fillna("Kategorie_NaN")

In [ ]:
# En se basant sur les connaissances du domaine, les caractéristiques demandées sont ajoutées.
# Toutes les caractéristiques nouvellement ajoutées (manuellement) recevront le préfixe "AppDK".

#Ratio par rapport à AMT_CREDIT.
dfapptraintest['AppDK_ANNUITY_CREDIT_RATIO'] = dfapptraintest['AMT_ANNUITY']/(dfapptraintest['AMT_CREDIT']+ 0.00001)
dfapptraintest['AppDK_GOODS_CREDIT_RATIO'] = dfapptraintest['AMT_GOODS_PRICE']/(dfapptraintest['AMT_CREDIT']+ 0.00001)

# Ratio par rapport à AMT_INCOME_TOTAL.
dfapptraintest['AppDK_ANNUITY_INCOME_RATIO'] = dfapptraintest['AMT_ANNUITY']/(dfapptraintest['AMT_INCOME_TOTAL']+ 0.00001)
dfapptraintest['AppDK_CREDIT_INCOME_RATIO'] = dfapptraintest['AMT_CREDIT']/(dfapptraintest['AMT_INCOME_TOTAL']+ 0.00001)
dfapptraintest['AppDK_GOODS_INCOME_RATIO'] = dfapptraintest['AMT_GOODS_PRICE']/(dfapptraintest['AMT_INCOME_TOTAL']+ 0.00001)
dfapptraintest['AppDK_CNT_FAM_INCOME_RATIO'] = dfapptraintest['AMT_INCOME_TOTAL']/(dfapptraintest['CNT_FAM_MEMBERS']+ 0.00001)

# Ratios par rapport à DAYS_BIRTH et DAYS_EMPLOYED.
dfapptraintest['AppDK_EMPLOYED_BIRTH_RATIO'] = dfapptraintest['DAYS_EMPLOYED']/(dfapptraintest['DAYS_BIRTH']+ 0.00001)
dfapptraintest['AppDK_INCOME_EMPLOYED_RATIO'] = dfapptraintest['AMT_INCOME_TOTAL']/(dfapptraintest['DAYS_EMPLOYED']+ 0.00001)
dfapptraintest['AppDK_INCOME_BIRTH_RATIO'] = dfapptraintest['AMT_INCOME_TOTAL']/(dfapptraintest['DAYS_BIRTH']+ 0.00001)
dfapptraintest['AppDK_CAR_BIRTH_RATIO'] = dfapptraintest['OWN_CAR_AGE'] / (dfapptraintest['DAYS_BIRTH']+ 0.00001)
dfapptraintest['AppDK_CAR_EMPLOYED_RATIO'] = dfapptraintest['OWN_CAR_AGE'] / (dfapptraintest['DAYS_EMPLOYED']+ 0.00001)

In [ ]:
# Vérification des remplacements.
dfapptraintest.isnull().sum().sum()

### 4 b)

Il convient de veiller à ajouter des caractéristiques qui apportent une valeur informative supplémentaire et qui ne sont pas fortement corrélées avec les caractéristiques existantes, afin d'éviter le surajustement (overfitting). L'interprétabilité des modèles peut également en souffrir si les nouvelles caractéristiques introduites n'apportent pas d'informations distinctes. Il est important d'éviter cela.

Les matrices de corrélation permettent de garantir la qualité des nouvelles caractéristiques introduites.

<br>
<a id="A5"></a>
<hr style="border:1px solid black"> </hr>
<h1 style="font-size:18px; font-weight: bold;">Tâche A-5 : Encodage</h1>
<hr style="border:1px solid black"> </hr>

**Expliquez l'utilité des procédures d'encodage dans le contexte de l'apprentissage automatique supervisé. Décrivez à la fois la méthode d'encodage one-hot et la méthode d'encodage par étiquette (label encoding), et abordez pour chaque méthode un avantage et un inconvénient.**

**Ensuite, les caractéristiques catégorielles des trois ensembles de données agrégées de la tâche précédente doivent être préparées comme suit :**

  - **Pour toutes les caractéristiques ayant au plus deux catégories, un encodage par étiquette (label encoding) doit être effectué.**
  - **Pour toutes les caractéristiques avec plus de deux catégories, le codage one-hot sera effectué.**


De nombreux algorithmes d'apprentissage automatique nécessitent le traitement de valeurs numériques (une exception est par exemple LightGBM). Par conséquent, les valeurs catégorielles doivent être encodées numériquement avant d'être traitées. Pour cela, il existe plusieurs méthodes :

Dans la partie 2, les nombres de variables catégorielles ont déjà été déterminés pour les trois ensembles de données : 11 (application_train), 8 (previous_application) et 1 (bureau).

In [ ]:
dfapptraintest.head()

In [ ]:
# application_traintest:
labelenc = LabelEncoder()
count = 0
columns_to_exclude = ['TARGET', 'TYPE']

columns_OHE = []
for col in dfapptraintest:
    if (dfapptraintest[col].dtype == 'category') and (col not in columns_to_exclude):
        if len(list(dfapptraintest[col].unique())) <= 2:
               labelenc.fit(dfapptraintest[col])
               dfapptraintest[col] = labelenc.transform(dfapptraintest[col])
               count += 1
        else:
            columns_OHE.append(col)

print('Label encoding: Total for ' + str(count) + ' columns.')

dfapptraintest = pd.get_dummies(dfapptraintest, columns=columns_OHE)

In [ ]:
dfapptraintest.head()

In [ ]:
# previous_application:
labelenc = LabelEncoder()
count = 0

for col in dfprevapp_agg:
    if dfprevapp_agg[col].dtype == 'category':
        if len(list(dfprevapp_agg[col].unique())) <= 2:
               labelenc.fit(dfprevapp_agg[col])
               dfprevapp_agg[col] = labelenc.transform(dfprevapp_agg[col])
               count += 1

print('A total of ' + str(count) + ' columns were encoded.')

# Encodage one-hot pour les variables catégorielles restantes :
dfprevapp_agg = pd.get_dummies(dfprevapp_agg)

dfprevapp_agg.head()

In [ ]:
# Attention : Encodage par étiquettes sur l'ensemble de données agrégé ! Des divergences par rapport aux tests peuvent être attendues !
# bureau:
labelenc = LabelEncoder()
count = 0

for col in dfbureau_agg:
    if dfbureau_agg[col].dtype == 'category':
        if len(list(dfbureau_agg[col].unique())) <= 2:
               labelenc.fit(dfbureau_agg[col])
               dfbureau_agg[col] = labelenc.transform(dfbureau_agg[col])
               count += 1

print('A total of ' + str(count) + ' columns were encoded.')

# One-hot encoding pour les variables catégorielles restantes:
dfbureau_agg = pd.get_dummies(dfbureau_agg)

dfbureau_agg.head()

<br>
<a id="A6"></a>
<hr style="border:1px solid black"> </hr>
<h1 style="font-size:18px; font-weight:bold;">Tâche A-6 : Agrégation du jeu de données principal</h1>
<hr style="border:1px solid black"> </hr>

**Dans cette section, le jeu de données doit être créé pour la modélisation. Pour ce faire, On utilise les ensembles de données encodés provenant de la tâche précédente comme suit :**

- **Les données de bureau (sous forme agrégée) doivent être ajoutées à l'ensemble de données combiné des données d'entraînement et de test en utilisant une jointure gauche.**

- **Ensuite, les données de previous_application (sous forme agrégée) doivent être ajoutées aux données de l'étape précédente.**
- **Enfin, le nombre de lignes et de colonnes, ainsi que cinq colonnes aléatoires, du jeu de données final doivent être affichés.**

**En fusionnant le jeu de données, de nouvelles "valeurs manquantes" (missing values) apparaissent. Celles-ci doivent être remplies avec la valeur 0.**

**Enfin, le jeu de données doit être divisé en un ensemble d'entraînement et un ensemble de test. Les variables auxiliaires introduites doivent ensuite être supprimées.**

In [ ]:
# Ajout de dfbureau
dfapptraintest_tmp1 = dfapptraintest.join(dfbureau_agg, how='left', on='SK_ID_CURR', rsuffix='_bureau')
dfapptraintest_tmp1.head()

In [ ]:
# Ajout de dfprevapp
df_all = dfapptraintest_tmp1.join(dfprevapp_agg, how='left', on='SK_ID_CURR', rsuffix='_prev')
df_all.sample(5)

In [ ]:
df_all.shape

In [ ]:
df_all.dtypes

In [ ]:
# Vérifier les NaN (valeurs manquantes)
df_all.isna().sum().sum()

In [ ]:
# Changez le type de données de TARGET en int.
df_all = df_all.astype({'TARGET':'int'})

# Changez le type de données de TYPE en int, où la valeur TRAIN est codée comme 0 et la valeur TEST est codée comme 1.
df_all['TYPE'].replace("TRAIN", "0", inplace= True)
df_all['TYPE'].replace("TEST", "1", inplace= True)
df_all = df_all.astype({'TYPE':'int'})

# Maintenant que toutes les colonnes sont numériques, toutes les valeurs manquantes peuvent être remplacées par 0.
df_all = df_all.fillna(0)

# Il n'y a plus de valeurs manquantes désormais.
df_all.isna().sum().sum()

In [ ]:
df_train = df_all[df_all['TYPE'] == 0]
df_train = df_train.drop(['TYPE'], axis=1)
df_train.shape

In [ ]:
df_test = df_all[df_all['TYPE'] == 1]
df_test = df_test.drop(['TYPE'], axis=1)
df_test.shape

Contrôle de qualité : Les ensembles d'entraînement et de test contiennent à nouveau le même nombre de lignes que dans la tâc

<br>
<a id="A7"></a>
<hr style="border:1px solid black"> </hr>
<h1 style="font-size:18px; font-weight:bold;">Tâche A-9 : Sélection de caractéristiques à l'aide d'un modèle Random Forest</h1>
<hr style="border:1px solid black"> </hr>

**En alternative à la valeur d'information (IV), un modèle Random Forest avec 250 arbres et au moins 25 points de données par feuille doit être calculé. L'importance des caractéristiques (FI) doit être représentée graphiquement pour les 20 meilleures caractéristiques. [Nombre d'arbres, taille des feuilles et top 20 doublés]**

**Ensuite, le jeu de données d'entraînement doit être modifié de manière à ne contenir que les colonnes avec une importance de caractéristique (FI) supérieure ou égale à 0,001, en plus des colonnes TARGET et SK_ID_CURR [IV retirée].**

**_Remarque : À la fin de cette tâche, le jeu de données d'entraînement devrait contenir au moins 100 colonnes. Si ce n'est pas le cas, les seuils pour IV et FI doivent être ajustés pour obtenir au moins 100 colonnes._**

In [ ]:
# Entraînement du Random Forest
tic = time.time()
rf = RandomForestClassifier(n_estimators = 500, min_samples_leaf = 50, random_state = 42, n_jobs=-1)
X = df_train.loc[:, df_train.columns != 'TARGET']
y = df_train.loc[:, df_train.columns == 'TARGET']

rf.fit(X, y)
print("time (sec):" + "%6.0f" % (time.time() - tic))

In [ ]:
# Afficher l'importance des caractéristiques : Top 20
feat_importances = pd.Series(rf.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')

In [ ]:
feat_imp = pd.DataFrame({'Variable':feat_importances.index, 'FI': feat_importances.values})
feat_imp['keep_var'] = feat_imp.apply(lambda row_: (row_.FI >= 0.001), axis=1)
keep_columns = feat_imp[feat_imp['keep_var'] == True].reset_index().drop(['index', 'keep_var'], axis=1)
len(keep_columns)

In [ ]:
columns_to_keep = ['SK_ID_CURR', 'TARGET']

columns_to_keep.extend(keep_columns['Variable'].tolist())
print(columns_to_keep)

In [ ]:
df = df_train[columns_to_keep]

In [ ]:
df.to_csv('/kaggle/working/df_feat_imp.csv', index=False)
